In [25]:
from DataReader import DataReaderChallangePolimi2017
import implicit
from CB_asy import CB_asymmetric_cosine
from CF_IB_asy import CF_IB_asymmetric_cosine
from CF_UB_asy import CF_UB_asymmetric_cosine
from implicit.nearest_neighbours import (BM25Recommender, CosineRecommender,TFIDFRecommender, bm25_weight)
from implicit import als
from sklearn.preprocessing import normalize
import numpy as np
import pickle
import scipy.sparse as sp

evaluation = True
rebuild = False
dr = DataReaderChallangePolimi2017(evaluation=evaluation,rebuild=rebuild)

urm = dr.getURM_csr()
t_p = dr.target_playlists
t_t = dr.target_tracks

Date Reader Challange Polimi 2017
Loading data from ./DumpData/DataReaderTrainTestDump
Dataset train-test loaded


In [27]:
model = als.AlternatingLeastSquares( factors=100, regularization=0.01, dtype=np.float32,
                             use_native=True, use_cg=True,
                             iterations=15, calculate_training_loss=False, num_threads=0
                           )

In [28]:
model.fit(bm25_weight(dr.getICM_csr(artists=True,albums=False,tags=False)))

In [29]:
a = model.item_factors
b = model.user_factors

In [30]:
k = 5

rec = []
x,y,v = [],[],[]
for p in dr.target_playlists:
    r = np.dot(b,a[p].T)
    idx = np.argpartition(r, -k)[-k:]
    x.extend(np.full(k,p))
    y.extend(idx)
    v.extend(r[idx])

In [31]:
r = sp.coo_matrix((v, (x, y)), shape=(dr.n_playlists, dr.n_tracks)).tocsr()

In [ ]:
dr.evaluateMAP(r)